In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
import math
%matplotlib inline 

In [2]:
def generate_sets(size, players = None):
    def to_bin_vector(value):
        return np.array([(value >> i) & 1 for i in range(size)])
    
    if players is None:
        players = np.arange(size)
        
    banned_ids = np.delete(np.arange(size), players)
    
    return [tuple(to_bin_vector(n)) for n in range(2**size) if (to_bin_vector(n)[banned_ids] == 0).all()]

In [3]:
def is_intersects(lhs, rhs):
    return np.minimum(np.array(lhs) == 1, np.array(rhs) == 1).any()

def indexes_array(bitvector):
    return np.arange(bitvector.shape[0])[bitvector]

def is_supper_additive(size, prices):
    all_combinations = generate_sets(size)

    for combination in all_combinations:
        comb_price = prices[combination]

        player_ids = indexes_array(np.array(combination) == 1)

        subcombinations = np.array(generate_sets(size, player_ids))
        
        for i in range(len(subcombinations)):
            for j in range(i + 1, len(subcombinations)):
                lhs = tuple(subcombinations[i])
                rhs = tuple(subcombinations[j])
                
                if is_intersects(lhs, rhs):
                    continue
                
                if comb_price < prices[lhs] + prices[rhs]:
                    return (False, combination, [lhs, rhs])

    return (True, )

def bit_where(func, lhs, rhs):
    return np.where(func(lhs == 1, rhs == 1), 
                    np.ones(lhs.shape[0], dtype = int),
                    np.zeros(lhs.shape[0], dtype = int))

def is_convex(size, prices):
    all_combinations = generate_sets(size)

    for i in range(len(all_combinations)):
        for j in range(i + 1, len(all_combinations)):
            lhs = np.array(all_combinations[i])
            rhs = np.array(all_combinations[j])
            
            intersection = tuple(bit_where(np.minimum, lhs, rhs))
            union = tuple(bit_where(np.maximum, lhs, rhs))

            if prices[union] + prices[intersection] < prices[tuple(lhs)] + prices[tuple(rhs)]:
                print(prices[union] + prices[intersection], prices[tuple(lhs)] + prices[tuple(rhs)])
                return (False, intersection, union, tuple(lhs), tuple(rhs))
                
    return (True, )


def shaply_vector(size, prices, player):
    sets = generate_sets(size)
    
    summary = 0
    
    for S in sets:
        if S[player] != 0:
            summary += math.factorial(np.sum(S) - 1) * math.factorial(size - np.sum(S)) \
                * (prices[S] - prices[tuple(np.where(np.arange(len(S)) == player, 0, S))])
            
    return (1 / math.factorial(size)) * summary

In [4]:
a1 = np.array([0, 1, 1])
a2 = np.array([0, 0, 1])
np.intersect1d(a1, a2)

array([0, 1])

In [5]:
prices_var15 = {
    (0, 0, 0, 0) : 0,
    (1, 0, 0, 0) : 2,
    (0, 1, 0, 0) : 3,
    (1, 1, 0, 0) : 6,
    (0, 0, 1, 0) : 2,
    (1, 0, 1, 0) : 5,
    (0, 1, 1, 0) : 6,
    (1, 1, 1, 0) : 9,
    (0, 0, 0, 1) : 4,
    (1, 0, 0, 1) : 7,
    (0, 1, 0, 1) : 8,
    (1, 1, 0, 1) : 11,
    (0, 0, 1, 1) : 8,
    (1, 0, 1, 1) : 10,
    (0, 1, 1, 1) : 12,
    (1, 1, 1, 1) : 14,
}

In [6]:
is_supper_additive(4, prices_var15)

(True,)

In [7]:
conv = is_convex(4, prices_var15)

print(conv)

for i in range(1, 5):
    print(prices_var15[conv[i]])
    
print()
if conv[0]:
    print('Is convex')
else:
    print('Not convex')

17 18
(False, (0, 1, 0, 0), (1, 1, 1, 1), (1, 1, 0, 0), (0, 1, 1, 1))
3
14
6
12

Not convex


In [8]:
vec = [shaply_vector(4, prices_var15, i) for i in range(4)]

print(np.round(vec, 3))
print(round(np.sum(vec), 3))

[2.417 3.75  2.917 4.917]
14.0
